<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2

Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Тема 5. Композиции алгоритмов, случайный лес
## <center>Практика. Деревья решений и случайный лес в соревновании Kaggle Inclass по кредитному скорингу

Тут веб-формы для ответов нет, ориентируйтесь на рейтинг [соревнования](https://inclass.kaggle.com/c/beeline-credit-scoring-competition-2), [ссылка](https://www.kaggle.com/t/115237dd8c5e4092a219a0c12bf66fc6) для участия.

Решается задача кредитного скоринга. 

Признаки клиентов банка:
- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

In [50]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

**Загружаем данные.**

In [51]:
train_df = pd.read_csv('../../data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('../../data/credit_scoring_test.csv', index_col='client_id')

In [52]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [53]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


**Посмотрим на число пропусков в каждом признаке.**

In [54]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          73084 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60153 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


In [55]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 75000 to 149999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          72992 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60116 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


**Заменим пропуски медианными значениями.**

In [56]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

### Дерево решений без настройки параметров

**Обучите дерево решений максимальной глубины 3, используйте параметр random_state=17 для воспроизводимости результатов.**

In [57]:
first_tree = DecisionTreeClassifier(random_state=42, max_depth=3)
first_tree.fit(train_df, y)

DecisionTreeClassifier(max_depth=3, random_state=42)

In [58]:
y

client_id
0        0
1        0
2        0
3        0
4        0
        ..
74995    0
74996    0
74997    0
74998    0
74999    0
Name: Delinquent90, Length: 75000, dtype: int64

**Сделайте прогноз для тестовой выборки.**

In [59]:
first_tree_pred = first_tree.predict_proba(test_df)[:,1]

In [60]:
first_tree_pred

array([0.02090472, 0.28203575, 0.02090472, ..., 0.09396285, 0.02090472,
       0.09396285])

**Запишем прогноз в файл.**

In [61]:
def write_to_submission_file(predicted_labels, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(75000, 
                                                  predicted_labels.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [62]:
write_to_submission_file(first_tree_pred, 'credit_scoring_first_tree.csv')

## Дерево решений с настройкой параметров с помощью GridSearch

**Настройте параметры дерева с помощью `GridSearhCV`, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр `random_state=17` (для воспроизводимости результатов), не забывайте про распараллеливание (`n_jobs=-1`).**

In [63]:
t = [1, 3, 5] + list(range(2, 20, 2))
# t = t.append(1)
print(t)

[1, 3, 5, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [66]:
%%time

tree_params = {'max_depth': list(range(2, 50, 5)), 
               'min_samples_leaf': [1, 3] + list(range(5, 10)) }

locally_best_tree = GridSearchCV(first_tree, tree_params, n_jobs=-1)
locally_best_tree.fit(train_df, y)


Wall time: 40.9 s


GridSearchCV(estimator=DecisionTreeClassifier(max_depth=3, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [2, 7, 12, 17, 22, 27, 32, 37, 42, 47],
                         'min_samples_leaf': [1, 3, 5, 6, 7, 8, 9]})

In [67]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

({'max_depth': 7, 'min_samples_leaf': 7}, 0.934)

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [68]:
tuned_tree_pred_probs = locally_best_tree.predict_proba(test_df)[:,1]

In [69]:
write_to_submission_file(tuned_tree_pred_probs, 'credit_scoring_sec_tree.csv')

### Случайный лес без настройки параметров

**Обучите случайный лес из деревьев неограниченной глубины, используйте параметр `random_state=17` для воспроизводимости результатов.**

In [70]:
 from sklearn.model_selection import train_test_split

In [71]:
X_train, X_val, y_train, y_val = train_test_split(train_df, y, train_size=0.8)

In [72]:
for i in range(2, 20, 2):
    first_forest = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=i)
    first_forest.fit(X_train, y_train)
    first_forest_pred = first_forest.predict(X_val)
    first_forest_pred_train = first_forest.predict(X_train)
    print(accuracy_score(first_forest_pred, y_val), accuracy_score(first_forest_pred_train, y_train))

0.9366666666666666 0.9322833333333334
0.9373333333333334 0.9334166666666667
0.939 0.9359833333333333
0.9394666666666667 0.9400333333333334
0.9394666666666667 0.9461166666666667
0.9386666666666666 0.953
0.9379333333333333 0.9595333333333333
0.9383333333333334 0.9663833333333334
0.9388666666666666 0.9741166666666666


In [73]:
first_forest_pred = first_forest.predict(X_val)
first_forest_pred_train = first_forest.predict(X_train)
accuracy_score(first_forest_pred, y_val), accuracy_score(first_forest_pred_train, y_train)

(0.9388666666666666, 0.9741166666666666)

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [74]:
write_to_submission_file(first_forest_pred, 'credit_scoring_sec_forest.csv')

### Случайный лес c настройкой параметров

**Настройте параметр `max_features` леса с помощью `GridSearhCV`, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр random_state=17 (для воспроизводимости результатов), не забывайте про распараллеливание (n_jobs=-1).**

In [75]:
GridSearchCV?

In [76]:
%%time
forest_params = {'max_features': np.linspace(0.3, 2, 10)}

best_forest = GridSearchCV(first_forest, forest_params, n_jobs=-1)
# print(best_forest)

Wall time: 6.9 ms


In [42]:
best_forest.fit(X_train, y_train)
best_forest_pred_train = first_forest.predict(X_train)
best_forest_pred = first_forest.predict(X_val)
print(accuracy_score(best_forest_pred, y_val), accuracy_score(first_forest_pred_train, y_train))

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py", line 441, in fit
    trees = Parallel(
  File "C:\Python39\lib\site-packages\joblib\parallel.py", line 1054, in __call__
    self.retrieve()
  File "C:\Python39\lib\site-packages\joblib\parallel.py", line 933, in retrieve
    self._output.extend(job.get(timeout=self.t

0.9328 0.9733


In [79]:
%%time
best_forest.fit(X_train, y_train).best_params_, round(best_forest.best_score_, 3)

KeyboardInterrupt: 

In [ ]:
tuned_forest_pred = locally_best_forest # Ваш код здесь

In [ ]:
write_to_submission_file # Ваш код здесь

**Посмотрите, как настроенный случайный лес оценивает важность признаков по их влиянию на целевой. Представьте результаты в наглядном виде с помощью `DataFrame`.**

In [ ]:
pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_ # Ваш код здесь

**Обычно увеличение количества деревьев только улучшает результат. Так что напоследок обучите случайный лес из 300 деревьев с найденными лучшими параметрами. Это может занять несколько минут.**

In [ ]:
%%time
final_forest = RandomForestClassifier # Ваш код здесь
final_forest.fit(train_df, y)
final_forest_pred = final_forest.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred, 'credit_scoring_final_forest.csv')

**Сделайте посылку на Kaggle.**